In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Revenue Forecast Model Comparisons

## Data

### Loading the data

In [3]:
#import raw revenue data
df_2016 = pd.read_csv("../../raw_data/orders2016.csv", sep=";")
df_2017 = pd.read_csv("../../raw_data/orders2017.csv", sep=";")
df_2018 = pd.read_csv("../../raw_data/orders2018.csv", sep=";")
df_2019 = pd.read_csv("../../raw_data/orders2019.csv", sep=";")
df_2020 = pd.read_csv("../../raw_data/orders2020.csv", sep=";")
df_2021 = pd.read_csv("../../raw_data/orders2021.csv", sep=";")
df_2022 = pd.read_csv("../../raw_data/orders2022.csv", sep=";")

df_list = [df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022]

#Dropping unnecessary columns, grouping by "date", summing "item_price" to get daily revenues

for i, df in enumerate(df_list):
    df_list[i] = pd.DataFrame(df.groupby(by="date")["item_price"].sum()/100)

#Concat all data in one dataframe, rename the columns for prophet

df = pd.concat(df_list, ignore_index=False)
df = df.rename(columns={"date": "ds", "item_price": "y"})
df["ds"] = df.index
df = df.reset_index(drop=True)
df = df[["ds","y"]]
df

#turning the ds (date) column into datetime

df['ds']=pd.to_datetime(df['ds'])

#Dropping outliers
df = df[df["y"]>=60]
df = df[df["y"]<=2300]
df = df.reset_index(drop=True)

In [5]:
#Loading additional regressor
feature_df = pd.read_csv("../../feature_data/final_feature_selection.csv")
feature_df["ds"] = pd.to_datetime(feature_df['ds'])

In [7]:
#Loading weather prediction data
weather_forecast = pd.read_csv("../../feature_data/finall_pred_weather.csv")
weather_forecast["ds"] = pd.to_datetime(weather_forecast["ds"])
weather_forecast["forecast dt iso"] = pd.to_datetime(weather_forecast["forecast dt iso"])

In [8]:
merged_df = pd.merge(df,feature_df,how="left")

In [9]:
merged_df.head()

,ds,y,temp,humidity,wind_speed,wind_deg,rain,clouds,Holiday,inflation_rate,Consumption Climate,cov_lock,unemp_Berlin_Mitte
0,2016-09-01,365.9,25.90,35,6.70,270,0.00,0,1,0.5,10.0,0,0.095
1,2016-09-02,358.6,22.18,46,4.63,280,0.00,75,1,0.5,10.0,0,0.095
2,2016-09-03,487.5,24.94,41,6.70,270,0.00,40,0,0.5,10.0,0,0.095
3,2016-09-04,90.9,19.94,73,6.70,230,0.00,75,0,0.5,10.0,0,0.095
4,2016-09-05,125.0,19.94,73,4.63,320,0.21,40,0,0.5,10.0,0,0.095


### Preparing data for different models

In [ ]:
#possible models: TimeSeries (Sarima, Arima, DNN), 

#1. introducing seasonal and weekly effects --> turning date into weekday information
#2. giving additional historical data --> revenue 1y ago, revenue past 1,2,3,7,14,30 days

## Splitting the data

In [6]:
#Setting variables
horizon = 16

#Splitting the data
split_date = "2022-08-01"
index_split = df[df["ds"]==split_date].index[0]
df_train = merged_df.iloc[:index_split]
df_test = merged_df.iloc[index_split:]
y_test = pd.DataFrame(df_test["y"])
weather_index_split = weather_forecast[weather_forecast["forecast dt iso"]==split_date].index[0]
weather_predict = weather_forecast.iloc[weather_index_split:weather_index_split+horizon,:]
weather_predict = weather_predict.drop(columns="forecast dt iso")

## Model

### Fitting

### Crossvalidating

### Predicting

In [ ]:
#Creating future dataframe
future = m.make_future_dataframe(periods=horizon)

In [ ]:
#Adding feature values to future dataframe
future = pd.merge(future,feature_df,how="left")
future

In [ ]:
#Update Future Timeframe with prediction weather data instead of historical weather data to prevent overfitting
cols_to_update = ['temp', 'humidity', 'clouds', 'wind_speed', 'wind_deg', 'rain']
future.loc[future.index[-(horizon):], cols_to_update] = weather_predict[cols_to_update].values

In [12]:
future

,ds,y,temp,humidity,wind_speed,wind_deg,rain,clouds,Holiday,inflation_rate,Consumption Climate,cov_lock,unemp_Berlin_Mitte
0,2016-09-01,365.9,25.90,35.0,6.70,270.0,0.00,0.0,1,0.5,10.0,0,0.095
1,2016-09-02,358.6,22.18,46.0,4.63,280.0,0.00,75.0,1,0.5,10.0,0,0.095
2,2016-09-03,487.5,24.94,41.0,6.70,270.0,0.00,40.0,0,0.5,10.0,0,0.095
3,2016-09-04,90.9,19.94,73.0,6.70,230.0,0.00,75.0,0,0.5,10.0,0,0.095
4,2016-09-05,125.0,19.94,73.0,4.63,320.0,0.21,40.0,0,0.5,10.0,0,0.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,2022-08-12,764.3,29.84,25.0,3.73,25.0,0.00,34.0,1,8.4,-36.8,0,0.091
1604,2022-08-13,800.7,29.12,29.0,4.52,31.0,0.00,18.0,1,8.4,-36.8,0,0.091
1605,2022-08-14,1103.1,27.84,37.0,4.61,53.0,0.00,97.0,1,8.4,-36.8,0,0.091
1606,2022-08-15,383.9,29.74,32.0,4.16,56.0,0.00,11.0,1,8.4,-36.8,0,0.091


In [ ]:
#Predicting
forecast = m.predict(future)

seven_day_forecast = forecast.tail(horizon)

seven_day_forecast_slim = seven_day_forecast[["ds","yhat_lower","yhat","yhat_upper"]]

prediction_forecast = seven_day_forecast_slim

prediction_forecast["y_true"] = y_test.head(horizon)

prediction_forecast["error"]=abs(prediction_forecast["yhat"]-prediction_forecast["y_true"])

prediction_forecast

### Evaluating

In [ ]:
#MAE
from sklearn.metrics import mean_absolute_error

y_true = prediction_forecast["y_true"]
y_pred = prediction_forecast["yhat"]

mae = mean_absolute_error(y_true, y_pred)

mae